In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import seaborn as sns
import datetime
from group_lasso import GroupLasso
from sklearn.utils import resample, check_random_state
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf
from tensorflow.keras import Sequential, Input, regularizers
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
import time
import pickle
import os
from numpy.random import seed
import lime
import lime.lime_tabular
seed(1)

from extra_functions import *

# Silence some warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('energydata_complete.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

fig = plot_data(df)
fig

### Generating extra features to describe time
weekday: number [0,6]\
weekstatus: binary describing weekend (1) or not (0)\
NSM: Number of Seconds from Midnight

These are used for filtering the data

In [ ]:
weekday = np.zeros(len(df))
weekstatus = np.zeros(len(df))
NSM = np.zeros(len(df))
month = np.zeros(len(df))

for i, val in enumerate(df.index):
    weekday[i] = val.weekday()
    weekstatus[i] = (weekday[i] >= 5)  # False for workday, True for weekend
    NSM[i] = (val - val.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    month[i] = val.month

df['weekday'] = weekday
#df['week status'] = weekstatus
df['NSM'] = NSM
df['month'] = month


## Add n previous timepoints to the data

Here we add the result vector from "t-n" as part of the covariates. 


In [ ]:
ns = [1, 6, 144] # Make a list in-case we want to skip some "n"
y = df['Appliances'].values # get y
for n in ns:
    temp = np.zeros_like(y)
    temp[n:] = y[:-n]
    df[f"t-{n}"]=temp
# Strip the first max(n) datapoints that now miss data
n = max(ns)
df = df[n:]

In [ ]:
plt.figure()

# These two plots should be identical
plt.plot(df['t-1'][:10],df['Appliances'][:10], lw=3, label="real")
plt.plot(df['t-1'][:10], df['t-1'][1:11], label="shifted t-1") 
plt.xlabel('t-1')
plt.ylabel('Appliances')
plt.legend()
plt.show()

### Filtering data and making training/validation/test set

In [ ]:
indices = (np.in1d(df.index.month, (1,2)))

# Train/validation/test
df_train = df[indices]
df_valid = df[(df.index.month==3)]
df_test = df[(df.index.month==4)]

X_train = np.array(df_train[df.columns[df.columns!='Appliances']])
y_train = np.array(df_train[df.columns[df.columns=='Appliances']])
X_test = np.array(df_test[df.columns[df.columns!='Appliances']])
y_test = np.array(df_test[df.columns[df.columns=='Appliances']])
X_valid = np.array(df_valid[df.columns[df.columns!='Appliances']])
y_valid = np.array(df_valid[df.columns[df.columns=='Appliances']])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
scaler = StandardScaler().fit(X_valid)
X_valid = scaler.transform(X_valid)
scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

y_test_mean = y_test.mean()
y_valid_mean = y_valid.mean()
y_train_mean = y_train.mean()

y_train = y_train - y_train_mean
y_valid = y_valid - y_valid_mean
y_test = y_test - y_test_mean

In [ ]:
# Check that the filter was correct
print(len(df[(df.index.month==1)])+ len(df[(df.index.month==2)]))
print(len(df_train))

### Correlations of covariates

In [ ]:
cor = df_train[df_train.columns].corr()
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(cor, square=True, xticklabels=True, yticklabels=True, cmap='RdBu')
plt.show()

## Neural Networks

### Feedforward Neural Network
The first neural network is actually not a RNN, but a feedforward network, where we include the input features t-n, which denotes the response variable from earlier time steps.

The network architecture consists of a number of dense layers each followed by a dropout layer.

### Hyper parameter tuning

Hyperparameter tuning of a NN model using either Random Search or Bayesian optimization:

The hyperparameters are:

1. Input layer size
2. Number of layers
3. Size of each layer
4. Regularization term (same for each layer)
5. Learning rate
6. Learning rate decay

The maximum values, minimum values and step size are chosen below in the method ```build_model```


# Next step might take a long time to run!!!

When ```trials = 1000``` the next step took 2 hours and 22 minutes to run.

In [ ]:
## Building NN and yuning hyper parameters

# Number of different models tried in the tuning process
trials = 2000
trials = 10
# Number of initial randomly selected points in the tuning process
starting_points = 200

# Early stopping
es = EarlyStopping(monitor="val_loss",patience=5)

# Timer
LOG_DIR = f"{int(time.time())}"

def build_model(hp):
    
    # Model
    model = Sequential()
    
    # Building layers
    for i in range(hp.Int('n_layers',min_value=1,max_value=3)):
        model.add(Dense(hp.Int(f"units_{i+1}",min_value=50,max_value=1000,step=50), 
             activation='relu', 
             name=f"dense{i+1}",
             kernel_regularizer=regularizers.l2(hp.Float('l2',1e-6,1)),           
            )
        )
        model.add(Dropout(hp.Float(f"drop_out_rate",0,0.9,0.1),
                         name=f"dropout{i+1}"))
    
    
    model.add(Dense(1,activation='linear', name="outputlayer"))

    # ADAM optimizer, learning rate and decay is specified
    opt = tf.keras.optimizers.Adam(lr=hp.Float('lr',1e-4,1e-3), decay=hp.Float('decay',1e-6,1e-5))

    # The NN is compiled. A MSE loss function is used.
    model.compile(loss='mse',
           optimizer=opt,
           metrics='mse',
           )
    
    return model

## Tuner object
# tuner = RandomSearch(build_model,
#                      objective = 'val_mse',
#                      max_trials = trials,
#                      executions_per_trial = 1,
#                      directory = LOG_DIR
#                     )

tuner = BayesianOptimization(build_model,
                     objective = 'val_mse',
                     max_trials = trials,
                     executions_per_trial = 1,
                     seed = 123,
                     num_initial_points = starting_points,
                     alpha = 1e-4,  # 1e-4 by default
                     beta = 1,   # 2.6 by default. I don't think we want to explore much and rather find a local min
                     directory = LOG_DIR
                    )

# Tuner search
tuner.search(x=X_train,
             y=y_train,
             epochs = 3,
             validation_data=(X_valid,y_valid))

In [ ]:
# Best selection of hyperparameters found from tuning
best_hp = tuner.get_best_hyperparameters()[0]
hp_folder = "best_hyper_params"
if not os.path.exists(hp_folder):
    os.mkdir(hp_folder)
filename = f"{hp_folder}/NN_hp_{trials}trials_{starting_points}init.pckl"
pickle.dump(best_hp, open(filename, 'wb'))
best_hp.values

In [ ]:
# Compiling the model using the best params
model = build_model(best_hp)

# The model is fitted to the training data and validated using the validation data. Variable number of epochs.
history = model.fit(X_train,y_train, epochs=20, verbose=0, validation_data=(X_valid,y_valid), callbacks=es)

# The predicted values are found using the test set
y_pred = model.predict(X_test)

In [ ]:
plt.figure()
plt.plot(np.arange(1,len(history.history['mse'])+1),history.history['mse'],'k-',label='Train')
plt.plot(np.arange(1,len(history.history['mse'])+1),history.history['val_mse'],'b-',label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Mean squared error')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# A summary of the used network
model.summary()

In [ ]:
# Function that calculates the R^2 value
def coeff_determination(y_true, y_pred):
    SS_res =  np.sum(( y_true - y_pred )**2)
    SS_tot = np.sum(( y_true - np.mean(y_true) )**2 )
    return ( 1 - SS_res/SS_tot )

# Train and test score given by R^2
print('Train score: '+ str(coeff_determination(y_train,model.predict(X_train))))
print('Test score: '+ str(coeff_determination(y_test,y_pred)))

In [ ]:
# A plot of real data and predicted data in time between two time intervals:

index_first = 1000
index_last = 1600

plt.figure(figsize=(12,8))
plt.plot(df_test.index[index_first:index_last],y_test[index_first:index_last]+y_test_mean,'k-', label='True')
plt.plot(df_test.index[index_first:index_last],y_pred[index_first:index_last]+y_test_mean,'r-', label='Predicted')
plt.legend()
plt.grid()
plt.xlabel('Date')
plt.ylabel('Appliances')
plt.show()

In [ ]:
# A plot of real data and predicted data in time between two time intervals:

index_first = 1100
index_last = 1110

plt.figure(figsize=(12,8))
plt.plot(df_test.index[index_first:index_last],y_test[index_first:index_last]+y_test_mean,'k-', label='True')
plt.plot(df_test.index[index_first:index_last],y_pred[index_first:index_last]+y_test_mean,'r-', label='Predicted')
plt.legend()
plt.grid()
plt.xlabel('Date')
plt.ylabel('Appliances')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(y_pred,y_test,'b.')
plt.plot([-100,900],[-100,900],'k-')
plt.grid()
plt.xlabel('Predicted Appliances')
plt.ylabel('True Appliances')
plt.show()

## Lime explaination of the model

In [ ]:
# add a predictor function that adds the y_test_mean
def predictor(*args, **kwargs):
    return model.predict(*args, **kwargs)+y_test_mean

explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=df.columns[df.columns!='Appliances'], 
                                                   class_names=['Appliances'],
                                                   categorical_features=['weekday'],
                                                   verbose=True, mode='regression')
# Pick a high consumption point
i = 1101
exp = explainer.explain_instance(X_test[i], predictor, num_features=10)

In [ ]:
exp.show_in_notebook(show_table=True)

In [ ]:
# Pick a average/low consumption point
i = 100
exp = explainer.explain_instance(X_test[i], predictor, num_features=10)
exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

## Decision tree surrogacy

Here we are going to investigate what the most important feature is in the trained neural net by training a surrugate model on the output of the NN model. Then we investigate the tree to figure out what features are most important for the NN predictions.

In [ ]:
# Sander's code here

# Retrain the model
# Compiling the model using the best params
model = build_model(best_hp)

# The model is fitted to the training data and validated using the validation data. Variable number of epochs.
_ = model.fit(X_train,y_train, epochs=20, verbose=0, validation_data=(X_valid,y_valid), callbacks=es)


# Set X train to original, but replace Y to the response of the network
s_X_train = X_train
s_Y_train = model.predict(X_train)

s_X_test = X_test
s_Y_test = model.predict(X_test)

In [ ]:
clf = DecisionTreeRegressor(random_state=42)
clf.fit(s_X_train, s_Y_train)
y_hat_train = clf.predict(s_X_train)
y_hat_test = clf.predict(s_X_test)
train_err = np.average(np.sqrt((y_hat_train-s_Y_train)**2))
test_err = np.average(np.sqrt((y_hat_test-s_Y_test)**2))
print(f"Train MSE: {train_err}")
print(f"Test MSE: {test_err}")

In [ ]:
print(f"Train score: {coeff_determination(s_Y_train, y_hat_train)}")
print(f"Train score: {coeff_determination(s_Y_test, y_hat_test)}")

In [ ]:
# A plot of real data and predicted data in time between two time intervals:

index_first = 1000
index_last = 1600

plt.figure(figsize=(12,8))
plt.plot(df_test.index[index_first:index_last],s_Y_test[index_first:index_last]+y_test_mean,'r-', label='NN Predicted')
plt.plot(df_test.index[index_first:index_last],y_hat_test[index_first:index_last]+y_test_mean,'b-', label='surrogate Predicted')
plt.plot(df_test.index[index_first:index_last],y_test[index_first:index_last]+y_test_mean,'k-', label='True')

plt.legend()
plt.grid()
plt.xlabel('Date')
plt.ylabel('Appliances')
plt.show()

In [ ]:
# Here we print the features in order of imporance
feature_importances = clf.tree_.compute_feature_importances()
idx = np.argsort(feature_importances)[::-1]
keys = df.keys()[1:]
for i in idx:
    print(f"{keys[i]}: {feature_importances[i]}")

## Random forrest

In [ ]:
# Youngrong's code her

## A randomforest model is made. All the ranges of the parameters used for the model are specified.
## Hyperparameters are tunned by gridsearchCV on validation data(Month==3)

model = RandomForestRegressor()

# Range of the gridsearch for RF hyperparameters
param_grid = {
    'bootstrap': [True],
    'max_depth': [5, 10, 20],
    'max_features': ['auto','sqrt'],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 5],
    'n_estimators': [100, 300, 500, 1000]
}

### Grid search

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                               cv=3, n_jobs=-1, verbose=2)

# Validation with the data to find the best set of hyperparameters
grid_search.fit(X_valid,y_valid.ravel())

# A summary of the chosen pararmeters
print(grid_search.best_params_)

# The model is fitted to the training data and validated using the validation data.
rf = model.set_params(**grid_search.best_params_)

rf.fit(X_train, y_train.ravel())

### Random Search

In [ ]:
random_search = RandomizedSearchCV(estimator = model, param_distributions = param_grid, n_iter = 100, cv = 3, verbose=2, random_state=1, n_jobs = -1)

# Validation with the data to find the best set of hyperparameters
random_search.fit(X_valid,y_valid.ravel())

# A summary of the chosen pararmeters
print('Best parameter:' +str(random_search.best_params_))

# The model is fitted to the training data and validated using the validation data.
rf = model.set_params(**random_search.best_params_)

rf.fit(X_train, y_train.ravel())

### Load the RF model

In [ ]:
# save the model to disk
filename = 'RF_model.sav'
pickle.dump(rf, open(filename, 'wb'))
# load the model from disk
#rf = pickle.load(open(filename, 'rb'))

In [ ]:
# The predicted values are found using the test data
y_pred = rf.predict(X_test)

print('Train score: '+ str(coeff_determination(y_train.ravel(),rf.predict(X_train))))
print('Test score: '+ str(coeff_determination(y_test.ravel(),y_pred)))

In [ ]:
# A plot of real data and predicted data in time between two time intervals:

index_first = 500
index_last = 1000

plt.figure(figsize=(12,8))
plt.plot(df_test.index[index_first:index_last],y_test[index_first:index_last],'k-', label='True')
plt.plot(df_test.index[index_first:index_last],y_pred[index_first:index_last],'r-', label='Predicted')
plt.legend()
plt.grid()
plt.xlabel('Date')
plt.ylabel('Appliances')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(y_pred,y_test,'b.')
plt.plot([0,1000],[0,1000],'k-')
plt.grid()
plt.xlabel('Predicted Appliances')
plt.ylabel('True Appliances')
plt.show()